In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from solarimage.classifier.nn.dataset import DataSet
from solarimage.classifier.nn import trainer

In [13]:
mnist = input_data.read_data_sets("example/MNIST_data/", one_hot=True)

Extracting example/MNIST_data/train-images-idx3-ubyte.gz


Extracting example/MNIST_data/train-labels-idx1-ubyte.gz
Extracting example/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting example/MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
from tensorflow.python.framework import dtypes
mnist_dataset = DataSet(mnist.train.images, mnist.train.labels,reshape=False,dtype=dtypes.uint8)

In [17]:
loss_list, weights, bias = trainer.runner(data_set, 100)

In [10]:
# need a function to present results in realtime
# Test trained model
with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                        y_: mnist.test.labels}))

NameError: name 'y' is not defined